In [1]:
import pandas as pd
import numpy as np

In [2]:
!wget https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv

--2023-01-21 17:42:18--  https://raw.githubusercontent.com/conversationai/unhealthy-conversations/main/unhealthy_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35005739 (33M) [text/plain]
Saving to: ‘unhealthy_full.csv.1’

unhealthy_full.csv. 100%[===================>]  33.38M  10.1MB/s    in 3.5s    

2023-01-21 17:42:22 (9.67 MB/s) - ‘unhealthy_full.csv.1’ saved [35005739/35005739]



In [2]:
data = pd.read_csv("unhealthy_full.csv")

In [3]:
ann_columns = data.columns.tolist()[4:]

In [4]:
', '.join(ann_columns)

'antagonize, condescending, dismissive, generalisation, generalisation_unfair, healthy, hostile, sarcastic'

In [5]:
for col in ann_columns:
    data[col] = data[col].astype(int)

In [9]:
np.random.seed(22)

selected_texts = data['comment'].drop_duplicates().sample(n=1000).tolist()
selected_data = data.loc[data['comment'].isin(selected_texts)].sample(n=1000)
not_selected_data = data.loc[~data['comment'].isin(selected_texts)]

In [10]:
mean_stds = not_selected_data.groupby('comment')[ann_columns].std().mean(axis=1).reset_index().rename(columns={0: 'mean_std'})
not_selected_data = not_selected_data.merge(mean_stds)

In [11]:

def get_personalized_prompt(rows):
    pattern = "Knowing that a certain person described the texts with given attributes:\n\n"
    labels = rows['labels'].apply(lambda x: ', '.join(x))
    comments = rows['comment']

    return pattern + '\n'.join(("Text: " + comments + '; Attributes: ' + labels).to_list())

def get_prompt(text, _worker_id):
    not_selected_worker_annotations = not_selected_data.loc[not_selected_data._worker_id == _worker_id]
    not_selected_worker_annotations = not_selected_worker_annotations.sort_values(by='mean_std', ascending=False)[:3]

    not_selected_worker_annotations['labels'] = not_selected_worker_annotations.apply(get_labels, axis=1)

    personalized_part = get_personalized_prompt(not_selected_worker_annotations)
    prompt = f'\n\nwrite which of the attributes: "antagonize", "condescending", "dismissive", "generalisation", "unfair generalisation", "healthy", "hostile", "sarcastic" this person described a given text? Write your answer in the form of a Python list containing the appropriate attributes \n\n Text: {text}\n\n'

    return personalized_part + prompt

def get_labels(row):
    return np.array(ann_columns)[row[ann_columns].values == 1]

selected_data['text_raw'] = selected_data['comment']
selected_data['comment'] = selected_data.apply(lambda row: get_prompt(row['comment'], row['_worker_id']), axis=1)

In [17]:
selected_data['labels'] = selected_data.apply(get_labels, axis=1)
selected_data.loc[:, ['comment', 'labels', 'text_raw']].to_csv('uc_personalized_prompts.csv', index=False)

In [18]:
!pwd

/mnt/data/mgruza/personalized-nlp
